In [1]:
import urllib.request
import re
import mysql.connector as connection

In [2]:
url = "http://archive.ics.uci.edu/ml/machine-learning-databases/00448/carbon_nanotubes.csv"

In [3]:
file = urllib.request.urlopen(url)

In [4]:
mydb = connection.connect(host="localhost",user="root", passwd="mysql123", database = "testdb1", use_pure=True)

In [5]:
i = 0
for line in file:
    #print(line)
    
    if i == 0:
        header_columns = line.decode('utf-8')
        #print(header_columns)
        
        header_cols = header_columns.split(";")
        
        for i in range(len(header_cols)):
            header_cols[i] = header_cols[i].split("'")[0]
            
        for i in range(len(header_cols)):
            header_cols[i] = re.sub(" ", "_", header_cols[i])
            if re.findall("indice", header_cols[i]):
                header_cols[i] += " int(4)"
            else:
                header_cols[i] += " varchar(20)"
        
        create_query = "create table if not exists carbon_nanotubes(" + "%s ," * len(header_cols)

        create_query = re.split(",$", create_query)[0]

        create_query += ")"

        create_query = create_query % tuple(header_cols)

        cur = mydb.cursor()
        cur.execute(create_query)
        
        mydb.commit()
        
        i += 1
    
    else:
        
        line_decode = line.decode('utf-8')
        
        line_elements_list = line_decode.split(";")
        
        for i in range(len(line_elements_list)):
            line_elements_list[i] = line_elements_list[i].split("\r")[0]
        
        insert_val = ""
        for i in line_elements_list:
            if i.isnumeric():
                insert_val += i
            else:
                insert_val += "'"
                insert_val += i
                insert_val += "'"

            insert_val += ", "
        
        insert_val = re.split(", $", insert_val)[0]
        
        cur = mydb.cursor()
        cur.execute("insert into carbon_nanotubes values ( %s )" %(insert_val))

mydb.commit()

In [6]:
cur = mydb.cursor()
cur.execute("select count(*) from carbon_nanotubes")
cur.fetchall()

[(10721,)]

In [7]:
cur = mydb.cursor()
cur.execute("select * from carbon_nanotubes limit 100")
cur.fetchall()

[(2,
  1,
  '0,679005',
  '0,701318',
  '0,017033',
  '0,721039',
  '0,730232',
  '0,017014'),
 (2, 1, '0,717298', '0,642129', '0,231319', '0,738414', '0,65675', '0,232369'),
 (2,
  1,
  '0,489336',
  '0,303751',
  '0,088462',
  '0,477676',
  '0,263221',
  '0,088712'),
 (2,
  1,
  '0,413957',
  '0,632996',
  '0,040843',
  '0,408823',
  '0,657897',
  '0,039796'),
 (2, 1, '0,334292', '0,543401', '0,15989', '0,303349', '0,558807', '0,157373'),
 (2, 1, '0,510664', '0,696249', '0,255128', '0,496977', '0,725608', '0,25597'),
 (2,
  1,
  '0,413957',
  '0,632996',
  '0,374176',
  '0,390878',
  '0,657645',
  '0,374735'),
 (2, 1, '0,712552', '0,55479', '0,112271', '0,750056', '0,561494', '0,114944'),
 (2, 1, '0,320995', '0,298682', '0,1837', '0,281301', '0,261433', '0,182779'),
 (2, 1, '0,394742', '0,279367', '0,302747', '0,35086', '0,2335', '0,302196'),
 (2,
  1,
  '0,282702',
  '0,357871',
  '0,397985',
  '0,229138',
  '0,328729',
  '0,398581'),
 (2, 1, '0,320995', '0,298682', '0,517033', '0,2